In [1]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text
import sys
print(sys.executable)
import findspark
findspark.init()
print(findspark.find())

import os
import sys
import json
import time
import pymongo
import certifi
import shutil
import pandas as pd

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window as W

/opt/anaconda3/envs/pysparkenv/bin/python
/opt/anaconda3/envs/pysparkenv/lib/python3.12/site-packages/pyspark


In [2]:
#Passing args for mysql and mongodb.
# For pandas 
mysql_pandas_args = {
    "uid" : "root",
    "pwd" : "mango4Pickle#",
    "hostname" : "localhost",
    "dbname" : "adventureworks"
}

# For Spark 
mysql_spark_args = {
    "hostname" : "localhost",
    "port" : "3306",
    "dbname" : "adventureworks",
    "conn_props" : {
        "user" : "root",
        "password" : "mango4Pickle#",
        "driver" : "com.mysql.cj.jdbc.Driver"
    }
} 
mongodb_args = {
    "user_name" : "arya_rajesh",
    "password" : "mango4Pickle#",
    "cluster_name" : "cluster0",
    "cluster_subnet" : "wsz4m57",
    "cluster_location" : "atlas", # "local"
    "db_name" : "adventureworks"
}

In [3]:
# Defining MongoDB get and set functions.

def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [4]:
# defining SQL get and set functions.
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe
    

def set_sql_datamart_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()
    
    
def wait_until_stream_is_ready(query, min_batches=1):
    while len(query.recentProgress) < min_batches:
        time.sleep(5)
        
    print(f"The stream has processed {len(query.recentProgress)} batchs")


In [5]:
#spark functions
def get_file_info(path: str):
    file_sizes = []
    modification_times = []

    '''Fetch each item in the directory, and filter out any directories.'''
    items = os.listdir(path)
    files = sorted([item for item in items if os.path.isfile(os.path.join(path, item))])

    '''Populate lists with the Size and Last Modification DateTime for each file in the directory.'''
    for file in files:
        file_sizes.append(os.path.getsize(os.path.join(path, file)))
        modification_times.append(pd.to_datetime(os.path.getmtime(os.path.join(path, file)), unit='s'))

    data = list(zip(files, file_sizes, modification_times))
    column_names = ['name','size','modification_time']
    
    return pd.DataFrame(data=data, columns=column_names)


def wait_until_stream_is_ready(query, min_batches=1):
    while len(query.recentProgress) < min_batches:
        time.sleep(5)
        
    print(f"The stream has processed {len(query.recentProgress)} batchs")


def remove_directory_tree(path: str):
    '''If it exists, remove the entire contents of a directory structure at a given 'path' parameter's location.'''
    try:
        if os.path.exists(path):
            shutil.rmtree(path)
            return f"Directory '{path}' has been removed successfully."
        else:
            return f"Directory '{path}' does not exist."
            
    except Exception as e:
        return f"An error occurred: {e}"
        

def drop_null_columns(df, threshold):
    '''Drop Columns having a percentage of NULL values that exceeds the given 'threshold' parameter value.'''
    columns_with_nulls = [col for col in df.columns if df.filter(df[col].isNull()).count() / df.count() > threshold] 
    df_dropped = df.drop(*columns_with_nulls) 
    
    return df_dropped
    
    
def get_mysql_dataframe(spark_session, sql_query : str, **args):
    '''Create a JDBC URL to the MySQL Database'''
    jdbc_url = f"jdbc:mysql://{args['hostname']}:{args['port']}/{args['dbname']}"
    
    '''Invoke the spark.read.format("jdbc") function to query the database, and fill a DataFrame.'''
    dframe = spark_session.read.format("jdbc") \
    .option("url", jdbc_url) \
    .option("driver", args['conn_props']['driver']) \
    .option("user", args['conn_props']['user']) \
    .option("password", args['conn_props']['password']) \
    .option("query", sql_query) \
    .load()
    
    return dframe
    
def get_mongo_uri(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the 'cluster_location' parameter.")
        
    if args['cluster_location'] == "atlas":
        uri = f"mongodb+srv://{args['user_name']}:{args['password']}@"
        uri += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net/"
    else:
        uri = "mongodb://localhost:27017/"

    return uri

def get_spark_conf_args(spark_jars : list, **args):
    jars = ""
    for jar in spark_jars:
        jars += f"{jar}, "
    
    sparkConf_args = {
        "app_name" : "PySpark Northwind Data Lakehouse (Medallion Architecture)",
        "worker_threads" : f"local[{int(os.cpu_count()/2)}]",
        "shuffle_partitions" : int(os.cpu_count()),
        "mongo_uri" : get_mongo_uri(**args),
        "spark_jars" : jars[0:-2],
        "database_dir" : sql_warehouse_dir
    }
    
    return sparkConf_args
    

def get_spark_conf(**args):
    sparkConf = SparkConf().setAppName(args['app_name'])\
    .setMaster(args['worker_threads']) \
    .set('spark.driver.memory', '4g') \
    .set('spark.executor.memory', '2g') \
    .set('spark.jars', args['spark_jars']) \
    .set('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
    .set('spark.mongodb.input.uri', args['mongo_uri']) \
    .set('spark.mongodb.output.uri', args['mongo_uri']) \
    .set('spark.sql.adaptive.enabled', 'false') \
    .set('spark.sql.debug.maxToStringFields', 35) \
    .set('spark.sql.shuffle.partitions', args['shuffle_partitions']) \
    .set('spark.sql.streaming.forceDeleteTempCheckpointLocation', 'true') \
    .set('spark.sql.streaming.schemaInference', 'true') \
    .set('spark.sql.warehouse.dir', args['database_dir']) \
    .set('spark.streaming.stopGracefullyOnShutdown', 'true')
    
    return sparkConf

In [6]:
# Dimension tables: dim_products, dim_employees, and dim_vendors and dim_date
# fact_purchaseorders fact table
# purchase orders through MongoDB, products through csv, vendor and dim_date through mysql

In [7]:
# read products csv and create df
base_dir = os.path.join(os.getcwd(), 'data')
batch_dir = os.path.join(base_dir, 'batch')
stream_dir = os.path.join(base_dir, 'streaming')
purchase_orders_stream_dir = os.path.join(stream_dir, 'purchase_orders')

data_dir = os.path.join(os.getcwd(), 'data')
data_file = os.path.join(batch_dir, 'adventureworks_products.csv')
df_products = pd.read_csv(data_file, header=0, index_col=0)
df_products.rename(columns={"ProductID":"product_key"}, inplace=True)
df_products.head(6)



dest_database = "adventureworks_dlh"
sql_warehouse_dir = os.path.abspath('spark-warehouse')
dest_database_dir = f"{dest_database}.db"
database_dir = os.path.join(sql_warehouse_dir, dest_database_dir)

purchase_orders_output_bronze = os.path.join(database_dir, 'fact_purchase_orders', 'bronze')
purchase_orders_output_silver = os.path.join(database_dir, 'fact_purchase_orders', 'silver')
purchase_orders_output_gold = os.path.join(database_dir, 'fact_purchase_orders', 'gold')

In [8]:
# create purchase_orders in MongoDB and create df
client = get_mongo_client(**mongodb_args)

data_dir = os.path.join(os.getcwd(), 'data')
#concatenating a string to get current working directory
json_files = {"purchase_orders" : 'adventureworks_purchaseorders.json'}

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)   

query = {} 
collection = "purchase_orders"
df_purchaseorders = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_purchaseorders.rename(columns={"VendorID":"vendor_key","ProductID":"product_key" }, inplace=True)

print(df_purchaseorders.head(2))

   PurchaseOrderID  RevisionNumber  Status  EmployeeID  vendor_key  \
0                1               0       4         244          83   
1                2               0       1         231          32   

   product_key  OrderQty  UnitPrice  LineTotal            OrderDate  ...  \
0            1         4    50.2600   201.0400  2001-05-17 00:00:00  ...   
1          360         3    45.5805   136.7415  2001-05-17 00:00:00  ...   

  ShipRate             ShipDate  SubTotal   TaxAmt  Freight  TotalDue  \
0     2.99  2001-05-26 00:00:00  201.0400  16.0832   5.0260  222.1492   
1     1.49  2001-05-26 00:00:00  272.1015  21.7681   6.8025  300.6721   

               DueDate  ReceivedQty RejectedQty  StockedQty  
0  2001-05-31 00:00:00          3.0         0.0         3.0  
1  2001-05-31 00:00:00          3.0         0.0         3.0  

[2 rows x 22 columns]


In [9]:
# get vendors from MySQL and create df
sql_vendor = "SELECT * FROM adventureworks.dim_vendors_vw;"
df_vendor = get_sql_dataframe(sql_vendor, **mysql_pandas_args)
df_vendor.rename(columns={"VendorID":"vendor_key"}, inplace=True)
df_vendor.head(2)

,vendor_key,AccountNumber,Name,CreditRating,PreferredVendorStatus,ActiveFlag,AddressType,AddressLine1,AddressLine2,City,StateProvinceCode,State_Province,PostalCode
0,1,INTERNAT0001,International,1,b'\x01',b'\x01',Main Office,683 Larch Ct.,None,Salt Lake City,UT,Utah,84101
1,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,b'\x01',b'\x01',Main Office,8547 Catherine Way,None,Tacoma,WA,Washington,98403


In [10]:
import os

os.listdir(purchase_orders_stream_dir)

['po_batch1.json', 'po_batch2.json', 'po_batch3.json']

In [11]:
# Found minimum date and maximum date on MySQL: 2001-05-17 and 2004-09-03. Manual calculated split into three.
sql_po_batch1 = "SELECT * FROM adventureworks.fact_purchase_orders_vw WHERE OrderDate < '2002-01-01';"

sql_po_batch2 = "SELECT * FROM adventureworks.fact_purchase_orders_vw WHERE OrderDate >= '2002-01-01' \
AND OrderDate < '2003-01-01';"

sql_po_batch3 = "SELECT * FROM adventureworks.fact_purchase_orders_vw WHERE OrderDate >= '2003-01-01';"

# Splitting into three batches 
df_po_1 = get_sql_dataframe(sql_po_batch1, **mysql_pandas_args)
df_po_2 = get_sql_dataframe(sql_po_batch2, **mysql_pandas_args)
df_po_3 = get_sql_dataframe(sql_po_batch3, **mysql_pandas_args)

#Exporting as JSON
df_po_1.to_json(
    os.path.join(purchase_orders_stream_dir, "po_batch1.json"),
    orient="records",
    lines=True
)

df_po_2.to_json(
    os.path.join(purchase_orders_stream_dir, "po_batch2.json"),
    orient="records",
    lines=True
)

df_po_3.to_json(
    os.path.join(purchase_orders_stream_dir, "po_batch3.json"),
    orient="records",
    lines=True
)

In [12]:
# creating and running new spark session
worker_threads = f"local[{int(os.cpu_count()/2)}]"

jars = []
mysql_spark_jar = os.path.join(os.getcwd(), "mysql-connector-j-9.1.0", "mysql-connector-j-9.1.0.jar")
mssql_spark_jar = os.path.join(os.getcwd(), "sqljdbc_12.8", "enu", "jars", "mssql-jdbc-12.8.1.jre11.jar")

jars.append(mysql_spark_jar)
#jars.append(mssql_spark_jar)

sparkConf_args = get_spark_conf_args(jars, **mongodb_args)

sparkConf = get_spark_conf(**sparkConf_args)
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
spark.sparkContext.setLogLevel("OFF")
spark

25/12/18 11:44:28 WARN Utils: Your hostname, MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 172.25.76.59 instead (on interface en0)
25/12/18 11:44:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/aryarajesh/.ivy2/cache
The jars for the packages stored in: /Users/aryarajesh/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4395f287-b62f-4b90-a653-320b82a5af21;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 60ms :: artifacts dl 3ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sy

:: loading settings :: url = jar:file:/opt/anaconda3/envs/pysparkenv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


25/12/18 11:44:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/18 11:44:28 WARN DependencyUtils: Local jar /Users/aryarajesh/Desktop/DS-2002-main/mysql-connector-j-9.1.0/mysql-connector-j-9.1.0.jar does not exist, skipping.
25/12/18 11:44:28 INFO SparkContext: Running Spark version 3.5.7
25/12/18 11:44:28 INFO SparkContext: OS info Mac OS X, 15.5, aarch64
25/12/18 11:44:28 INFO SparkContext: Java version 21.0.9
25/12/18 11:44:28 INFO ResourceUtils: ==============================================================
25/12/18 11:44:28 INFO ResourceUtils: No custom resources configured for spark.driver.
25/12/18 11:44:28 INFO ResourceUtils: ==============================================================
25/12/18 11:44:28 INFO SparkContext: Submitted application: PySpark Northwind Data Lakehouse (Medallion Architecture)
25/12/18 11:44:28 INFO ResourceProfile: Default ResourceProfile created, executor resources:

In [13]:
#creating new metadata base
spark.sql(f"DROP DATABASE IF EXISTS {dest_database} CASCADE;")

sql_create_db = f"""
    CREATE DATABASE IF NOT EXISTS {dest_database}
    COMMENT 'DS-2002 Final Project'
    WITH DBPROPERTIES (contains_pii = true, purpose = 'Final Project');
"""
spark.sql(sql_create_db)

DataFrame[]

In [14]:
# populating employee dimension.
employee_csv = os.path.join(batch_dir, 'adventureworks_employees.csv')
print(employee_csv)

df_dim_employees = spark.read.format('csv').options(header='true', inferSchema='true').load(employee_csv)
df_dim_employees.toPandas().head(2)

/Users/aryarajesh/Desktop/DS-2002-main/data/batch/adventureworks_employees.csv


,EmployeeID,NationalIDNumber,LoginID,ManagerID,FirstName,MiddleName,LastName,Title,EmailAddress,EmailPromotion,Phone,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag
0,1,14417807,adventure-works\guy1,16,Guy,R,Gilbert,Production Technician - WC60,guy1@adventure-works.com,0,320-555-0195,1972-05-15,M,M,1996-07-31,0,21,30,1
1,2,253022876,adventure-works\kevin0,6,Kevin,F,Brown,Marketing Assistant,kevin0@adventure-works.com,2,150-555-0189,1977-06-03,S,M,1997-02-26,0,42,41,1


In [15]:
# Renaming id. Adding the primary key column using
# SQL windowing. Reordering columns.
df_dim_employees = df_dim_employees.withColumnRenamed("EmployeeID", "employee_id")
# ----------------------------------------------------------------------------------
# Add Primary Key column using SQL Windowing function: ROW_NUMBER() 
# ----------------------------------------------------------------------------------
df_dim_employees.createOrReplaceTempView("employees")
sql_employees = f"""
    SELECT *, ROW_NUMBER() OVER (ORDER BY employee_id) AS employee_key
    FROM employees;
"""
df_dim_employees = spark.sql(sql_employees)

# ----------------------------------------------------------------------------------
# Reorder Columns and display the first two rows in a Pandas dataframe
# ----------------------------------------------------------------------------------
ordered_columns = ['employee_key', 'employee_id', 'FirstName', 'LastName', 'NationalIDNumber'
                  , 'Title', 'Phone']

df_dim_employees = df_dim_employees[ordered_columns]
df_dim_employees.toPandas().head(2)


,employee_key,employee_id,FirstName,LastName,NationalIDNumber,Title,Phone
0,1,1,Guy,Gilbert,14417807,Production Technician - WC60,320-555-0195
1,2,2,Kevin,Brown,253022876,Marketing Assistant,150-555-0189


In [16]:
#Fixing an error that keeps popping up for next cell:
spark.sql(f"DROP DATABASE IF EXISTS {dest_database} CASCADE")
spark.sql(f"""
CREATE DATABASE IF NOT EXISTS {dest_database}
COMMENT 'AdventureWorks Data Lakehouse'
""")

DataFrame[]

In [17]:
#Save in lakehouse.
spark.sql(f"DROP TABLE IF EXISTS {dest_database}.dim_employees")
df_dim_employees.write.saveAsTable(f"{dest_database}.dim_employees", mode="overwrite")

In [18]:
#Previewing employee table.
spark.sql(f"DESCRIBE EXTENDED {dest_database}.dim_employees;").show()
spark.sql(f"SELECT * FROM {dest_database}.dim_employees LIMIT 2").toPandas()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|        employee_key|                 int|   NULL|
|         employee_id|                 int|   NULL|
|           FirstName|              string|   NULL|
|            LastName|              string|   NULL|
|    NationalIDNumber|                 int|   NULL|
|               Title|              string|   NULL|
|               Phone|              string|   NULL|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|             Catalog|       spark_catalog|       |
|            Database|  adventureworks_dlh|       |
|               Table|       dim_employees|       |
|        Created Time|Thu Dec 18 11:44:...|       |
|         Last Access|             UNKNOWN|       |
|          Created By|         Spark 3.5.7|       |
|                Type|             MANAGED|       |
|           

,employee_key,employee_id,FirstName,LastName,NationalIDNumber,Title,Phone
0,1,1,Guy,Gilbert,14417807,Production Technician - WC60,320-555-0195
1,2,2,Kevin,Brown,253022876,Marketing Assistant,150-555-0189


In [19]:
#Populating products dimension
product_csv = os.path.join(batch_dir, "adventureworks_products.csv")

df_dim_products = (
    spark.read
    .format("csv")
    .options(header=True, inferSchema=True)
    .load(product_csv)
)

df_dim_products.show(5)
#renaming key
df_dim_products = df_dim_products.withColumnRenamed("ProductID", "product_id")
# Add Primary Key column using SQL Windowing function: ROW_NUMBER() 
# ----------------------------------------------------------------------------------
df_dim_products.createOrReplaceTempView("products")
sql_products = f"""
    SELECT *, ROW_NUMBER() OVER (ORDER BY product_id) AS product_key
    FROM products;
"""
df_dim_products = spark.sql(sql_products)

# ----------------------------------------------------------------------------------
# Reorder Columns and display the first two rows in a Pandas dataframe
# ----------------------------------------------------------------------------------
ordered_columns = ['product_key', 'product_id','Name', 'StandardCost']

df_dim_products = df_dim_products[ordered_columns]
df_dim_products.toPandas().head(2)


+---+---------+--------------------+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+---------------+------------------+------------+-------------+-----------+----------------+
|_c0|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag|Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductCategory|ProductSubcategory|ProductModel|SellStartDate|SellEndDate|DiscontinuedDate|
+---+---------+--------------------+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+---------------+------------------+------------+-------------+-----------+----------------+
|  1|        1|     Adjustab

,product_key,product_id,Name,StandardCost
0,1,1,Adjustable Race,0.0
1,2,2,Bearing Ball,0.0


In [20]:
#Save products in lakehouse.
#spark.sql(f"DROP TABLE IF EXISTS {dest_database}.dim_products")
df_dim_products.write.saveAsTable(f"{dest_database}.dim_products", mode="overwrite")

In [21]:
#Previewing products table.
spark.sql(f"DESCRIBE EXTENDED {dest_database}.dim_products;").show()
spark.sql(f"SELECT * FROM {dest_database}.dim_products LIMIT 2").toPandas()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|         product_key|                 int|   NULL|
|          product_id|                 int|   NULL|
|                Name|              string|   NULL|
|        StandardCost|              double|   NULL|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|             Catalog|       spark_catalog|       |
|            Database|  adventureworks_dlh|       |
|               Table|        dim_products|       |
|        Created Time|Thu Dec 18 11:44:...|       |
|         Last Access|             UNKNOWN|       |
|          Created By|         Spark 3.5.7|       |
|                Type|             MANAGED|       |
|            Provider|             parquet|       |
|            Location|file:/Users/aryar...|       |
+--------------------+--------------------+-------+



,product_key,product_id,Name,StandardCost
0,1,1,Adjustable Race,0.0
1,2,2,Bearing Ball,0.0


In [22]:
#Populating vendors dimension
sql_vendors = "SELECT * FROM adventureworks.dim_vendors_vw;"
pdf_vendors = get_sql_dataframe(sql_vendors, **mysql_pandas_args)

df_dim_vendors = spark.createDataFrame(pdf_vendors)
df_dim_vendors.show(5)

#renaming key
df_dim_vendors = df_dim_vendors.withColumnRenamed("VendorID", "vendor_id")
# Add Primary Key column using SQL Windowing function: ROW_NUMBER() 
# ----------------------------------------------------------------------------------
df_dim_vendors.createOrReplaceTempView("vendors")
sql_vendors = f"""
    SELECT *, ROW_NUMBER() OVER (ORDER BY vendor_id) AS vendor_key
    FROM vendors;
"""
df_dim_vendors = spark.sql(sql_vendors)

# ----------------------------------------------------------------------------------
# Reorder Columns and display the first two rows in a Pandas dataframe
# ----------------------------------------------------------------------------------
ordered_columns = ['vendor_key', 'vendor_id', 'Name', 'AccountNumber','AddressType', "AddressLine1", "City", "State_Province",'PostalCode' ]

df_dim_vendors = df_dim_vendors[ordered_columns]
df_dim_vendors.toPandas().head(2)

+--------+-------------+--------------------+------------+---------------------+----------+-----------+--------------------+------------+--------------+-----------------+--------------+----------+
|VendorID|AccountNumber|                Name|CreditRating|PreferredVendorStatus|ActiveFlag|AddressType|        AddressLine1|AddressLine2|          City|StateProvinceCode|State_Province|PostalCode|
+--------+-------------+--------------------+------------+---------------------+----------+-----------+--------------------+------------+--------------+-----------------+--------------+----------+
|       1| INTERNAT0001|       International|           1|                 [01]|      [01]|Main Office|       683 Larch Ct.|        NULL|Salt Lake City|              UT |          Utah|     84101|
|       2| ELECTRON0002|Electronic Bike R...|           1|                 [01]|      [01]|Main Office|  8547 Catherine Way|        NULL|        Tacoma|              WA |    Washington|     98403|
|       3|  PRE

,vendor_key,vendor_id,Name,AccountNumber,AddressType,AddressLine1,City,State_Province,PostalCode
0,1,1,International,INTERNAT0001,Main Office,683 Larch Ct.,Salt Lake City,Utah,84101
1,2,2,Electronic Bike Repair & Supplies,ELECTRON0002,Main Office,8547 Catherine Way,Tacoma,Washington,98403


In [23]:
#Save vendors in lakehouse.
#spark.sql(f"DROP TABLE IF EXISTS {dest_database}.dim_vendors")
df_dim_vendors.write.saveAsTable(f"{dest_database}.dim_vendors", mode="overwrite")

In [24]:
#Previewing vendors table.
spark.sql(f"DESCRIBE EXTENDED {dest_database}.dim_vendors;").show()
spark.sql(f"SELECT * FROM {dest_database}.dim_vendors LIMIT 2").toPandas()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|          vendor_key|                 int|   NULL|
|           vendor_id|              bigint|   NULL|
|                Name|              string|   NULL|
|       AccountNumber|              string|   NULL|
|         AddressType|              string|   NULL|
|        AddressLine1|              string|   NULL|
|                City|              string|   NULL|
|      State_Province|              string|   NULL|
|          PostalCode|              string|   NULL|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|             Catalog|       spark_catalog|       |
|            Database|  adventureworks_dlh|       |
|               Table|         dim_vendors|       |
|        Created Time|Thu Dec 18 11:44:...|       |
|         Last Access|             UNKNOWN|       |
|          C

,vendor_key,vendor_id,Name,AccountNumber,AddressType,AddressLine1,City,State_Province,PostalCode
0,1,1,International,INTERNAT0001,Main Office,683 Larch Ct.,Salt Lake City,Utah,84101
1,2,2,Electronic Bike Repair & Supplies,ELECTRON0002,Main Office,8547 Catherine Way,Tacoma,Washington,98403


In [25]:
# creating streaming purchase order data
sql_purchase_orders = "SELECT * FROM adventureworks.fact_purchase_orders_vw;"
df_purchase_orders = get_sql_dataframe(sql_purchase_orders, **mysql_pandas_args)

#minimizing 
df_purchase_orders = df_purchase_orders[[
    "PurchaseOrderID",
    "EmployeeID",
    "VendorID",
    "ProductID",
    "OrderDate",
    "ShipDate",
    "OrderQty",
    "UnitPrice",
    "LineTotal",
    "TotalDue"
]]

df_purchase_orders.head(2)

,PurchaseOrderID,EmployeeID,VendorID,ProductID,OrderDate,ShipDate,OrderQty,UnitPrice,LineTotal,TotalDue
0,1,244,83,1,2001-05-17,2001-05-26,4,50.2600,201.0400,222.1492
1,2,231,32,360,2001-05-17,2001-05-26,3,45.5805,136.7415,300.6721


In [26]:
#checking what dimensions are in the lakehouse
spark.sql(f"SHOW TABLES IN {dest_database}").show(truncate=False)

+------------------+-------------+-----------+
|namespace         |tableName    |isTemporary|
+------------------+-------------+-----------+
|adventureworks_dlh|dim_employees|false      |
|adventureworks_dlh|dim_products |false      |
|adventureworks_dlh|dim_vendors  |false      |
|                  |employees    |true       |
|                  |products     |true       |
|                  |vendors      |true       |
+------------------+-------------+-----------+



In [27]:
# creating dim_date df - edit for spark
sql_dim_date = "SELECT * FROM adventureworks.dim_date;"
df_dim_date = get_sql_dataframe(
    sql_dim_date,
    **mysql_pandas_args)
df_dim_date.head(2)

,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
1,20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


In [28]:
#create dim_date in lakehouse
df_dim_date_spark = spark.createDataFrame(df_dim_date)
df_dim_date_spark.show(5)
spark.sql(f"DROP TABLE IF EXISTS {dest_database}.dim_date")
df_dim_date_spark.write.saveAsTable(
    f"{dest_database}.dim_date",
    mode="overwrite"
)

+--------+----------+----------+------------+------------+-----------+----------------+------------+-----------+---------------+------------+----------+-------------+--------------------+----------------+-------------+-------------------+-----------------+--------------------+--------------+-----------+-----------------+---------------+
|date_key| full_date| date_name|date_name_us|date_name_eu|day_of_week|day_name_of_week|day_of_month|day_of_year|weekday_weekend|week_of_year|month_name|month_of_year|is_last_day_of_month|calendar_quarter|calendar_year|calendar_year_month|calendar_year_qtr|fiscal_month_of_year|fiscal_quarter|fiscal_year|fiscal_year_month|fiscal_year_qtr|
+--------+----------+----------+------------+------------+-----------+----------------+------------+-----------+---------------+------------+----------+-------------+--------------------+----------------+-------------+-------------------+-----------------+--------------------+--------------+-----------+-----------------+

In [29]:
spark.table(f"{dest_database}.dim_date").show(2)

+--------+----------+----------+------------+------------+-----------+----------------+------------+-----------+---------------+------------+----------+-------------+--------------------+----------------+-------------+-------------------+-----------------+--------------------+--------------+-----------+-----------------+---------------+
|date_key| full_date| date_name|date_name_us|date_name_eu|day_of_week|day_name_of_week|day_of_month|day_of_year|weekday_weekend|week_of_year|month_name|month_of_year|is_last_day_of_month|calendar_quarter|calendar_year|calendar_year_month|calendar_year_qtr|fiscal_month_of_year|fiscal_quarter|fiscal_year|fiscal_year_month|fiscal_year_qtr|
+--------+----------+----------+------------+------------+-----------+----------------+------------+-----------+---------------+------------+----------+-------------+--------------------+----------------+-------------+-------------------+-----------------+--------------------+--------------+-----------+-----------------+

In [30]:
# merging purchaseorders and vendor dfs
df_fact_purchaseorders = pd.merge(df_purchaseorders, df_vendor, on='vendor_key', how='inner')
df_fact_purchaseorders.rename(columns={"Name":"vendor_name"}, inplace=True)
df_fact_purchaseorders.drop(['EmployeeID'], axis=1, inplace=True)
df_fact_purchaseorders.head(2)

,PurchaseOrderID,RevisionNumber,Status,vendor_key,product_key,OrderQty,UnitPrice,LineTotal,OrderDate,ShipMethod,...,CreditRating,PreferredVendorStatus,ActiveFlag,AddressType,AddressLine1,AddressLine2,City,StateProvinceCode,State_Province,PostalCode
0,1,0,4,83,1,4,50.2600,201.0400,2001-05-17 00:00:00,OVERSEAS - DELUXE,...,1,b'\x01',b'\x01',Main Office,4405 Balboa Court,None,Santa Cruz,CA,California,95062
1,2,0,1,32,360,3,45.5805,136.7415,2001-05-17 00:00:00,CARGO TRANSPORT 5,...,1,b'\x01',b'\x01',Main Office,7995 Edwards Ave.,None,Lynnwood,WA,Washington,98036


In [31]:
# merging purchaseorders and product dfs
df_fact_purchaseorders = pd.merge(df_fact_purchaseorders, df_products, on='product_key', how='inner')
df_fact_purchaseorders.rename(columns={"Name":"product_name"}, inplace=True)
df_fact_purchaseorders.head(2)

,PurchaseOrderID,RevisionNumber,Status,vendor_key,product_key,OrderQty,UnitPrice,LineTotal,OrderDate,ShipMethod,...,DaysToManufacture,ProductLine,Class,Style,ProductCategory,ProductSubcategory,ProductModel,SellStartDate,SellEndDate,DiscontinuedDate
0,1,0,4,83,1,4,50.2600,201.0400,2001-05-17 00:00:00,OVERSEAS - DELUXE,...,0,NaN,NaN,NaN,NaN,NaN,NaN,6/1/98 0:00,NaN,NaN
1,2,0,1,32,360,3,45.5805,136.7415,2001-05-17 00:00:00,CARGO TRANSPORT 5,...,0,NaN,NaN,NaN,NaN,NaN,NaN,6/1/98 0:00,NaN,NaN


In [32]:
#merging purchaseorders and dim_date
df_fact_purchaseorders['order_date'] = pd.to_datetime(df_fact_purchaseorders['OrderDate']).astype('datetime64[ns]').dt.date
df_dim_order_date = df_dim_date.rename(columns={"date_key" : "order_date_key", "full_date" : "order_date"})
df_fact_purchaseorders.drop(['OrderDate'], axis=1, inplace=True)
df_fact_purchaseorders = pd.merge(df_fact_purchaseorders, df_dim_order_date, on='order_date', how='left')
df_fact_purchaseorders.head(2)

#Deleting Columns and Ordering Fact Table
df_fact_purchaseorders.drop(columns=['RevisionNumber','ShipBase','CreditRating','PreferredVendorStatus','MakeFlag',
 'FinishedGoodsFlag','Color','SafetyStockLevel','ReorderPoint','Size','SizeUnitMeasureCode','WeightUnitMeasureCode',
                                     'Weight','DaysToManufacture','ProductLine','Class','Style' ], axis=1, inplace=True)

desired_order = ['PurchaseOrderID',
  'order_date',
 'Status',
 'vendor_key',       
 'product_key', 
'order_date_key',
 'ProductNumber',
 'ProductCategory',
 'ProductSubcategory',
 'ProductModel',
 'OrderQty',
 'UnitPrice',
 'LineTotal',
 'ShipMethod',
 'ShipRate',
 'ShipDate',
 'SubTotal',
 'TaxAmt',
 'Freight',
 'TotalDue',
 'DueDate',
 'ReceivedQty',
 'RejectedQty',
 'StockedQty',
 'AccountNumber',
 'StandardCost',
 'ListPrice',
 'SellStartDate',
 'SellEndDate',
 'DiscontinuedDate',
]

In [33]:
# Order and check columns
df_fact_purchaseorders = df_fact_purchaseorders[desired_order]
list(df_fact_purchaseorders)
df_fact_purchaseorders.head(2)

,PurchaseOrderID,order_date,Status,vendor_key,product_key,order_date_key,ProductNumber,ProductCategory,ProductSubcategory,ProductModel,...,DueDate,ReceivedQty,RejectedQty,StockedQty,AccountNumber,StandardCost,ListPrice,SellStartDate,SellEndDate,DiscontinuedDate
0,1,2001-05-17,4,83,1,20010517,AR-5381,NaN,NaN,NaN,...,2001-05-31 00:00:00,3.0,0.0,3.0,LITWARE0001,0.0,0.0,6/1/98 0:00,NaN,NaN
1,2,2001-05-17,1,32,360,20010517,HJ-1220,NaN,NaN,NaN,...,2001-05-31 00:00:00,3.0,0.0,3.0,ADVANCED0001,0.0,0.0,6/1/98 0:00,NaN,NaN


In [34]:
df_fact_purchaseorders.insert(0, 'purchaseorder_key', range(1, len(df_fact_purchaseorders) + 1))

In [35]:
# Create new destination database (data mart)

mysql_datamart_args = {
    "uid" : "root",
    "pwd" : "mango4Pickle#",
    "hostname" : "localhost",
    "dbname" : "adventureworks_datamart"
}

conn_str = f"mysql+pymysql://{mysql_datamart_args['uid']}:{mysql_datamart_args['pwd']}@{mysql_datamart_args['hostname']}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{mysql_datamart_args['dbname']}`;"))
connection.execute(text(f"CREATE DATABASE `{mysql_datamart_args['dbname']}`;"))
connection.execute(text(f"USE {mysql_datamart_args['dbname']};"))

connection.close()

In [36]:
# Insert tables into data mart
db_operation = "insert"

tables = [('dim_vendor', df_vendor, 'vendor_key'),
          ('dim_products', df_products, 'product_key'),
          ('fact_purchaseorders', df_fact_purchaseorders, 'purchaseorder_key'),
         ('dim_date', df_dim_date, 'date_key')]
for table_name, dataframe, primary_key in tables:
    set_sql_datamart_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_datamart_args)


In [37]:
df_fact_purchaseorders.head(2)

,purchaseorder_key,PurchaseOrderID,order_date,Status,vendor_key,product_key,order_date_key,ProductNumber,ProductCategory,ProductSubcategory,...,DueDate,ReceivedQty,RejectedQty,StockedQty,AccountNumber,StandardCost,ListPrice,SellStartDate,SellEndDate,DiscontinuedDate
0,1,1,2001-05-17,4,83,1,20010517,AR-5381,NaN,NaN,...,2001-05-31 00:00:00,3.0,0.0,3.0,LITWARE0001,0.0,0.0,6/1/98 0:00,NaN,NaN
1,2,2,2001-05-17,1,32,360,20010517,HJ-1220,NaN,NaN,...,2001-05-31 00:00:00,3.0,0.0,3.0,ADVANCED0001,0.0,0.0,6/1/98 0:00,NaN,NaN


In [38]:
#Create analysis queries:

sql_fact_purchaseorders = """
SELECT 
dt.full_date, p.productnumber, v.accountnumber, po.purchaseorderid, po.order_date
FROM fact_purchaseorders AS po
INNER JOIN dim_vendor AS v ON v.vendor_key = po.vendor_key
INNER JOIN dim_products AS p ON p.product_key = po.product_key
INNER JOIN dim_date as dt ON dt.date_key = po.order_date_key
"""
df_datamart_purchaseorders = get_sql_dataframe(sql_fact_purchaseorders, **mysql_datamart_args)
df_datamart_purchaseorders.head(2)

,full_date,productnumber,accountnumber,purchaseorderid,order_date
0,2001-05-17,AR-5381,LITWARE0001,1,2001-05-17
1,2001-05-17,HJ-1220,ADVANCED0001,2,2001-05-17


In [39]:
#Query 1: Showing total amount due per vendor for each product
sql_fact_purchaseorders = """
SELECT 
v.name AS 'Vendor Name', 
p.productnumber AS 'Product Number',
SUM(po.totaldue) AS 'Total Due Amount'
FROM fact_purchaseorders AS po
INNER JOIN dim_vendor AS v ON v.vendor_key = po.vendor_key
INNER JOIN dim_products AS p ON p.product_key = po.product_key
INNER JOIN dim_date as dt ON dt.date_key = po.order_date_key
GROUP BY v.vendor_key, p.productnumber
ORDER BY v.name, p.productnumber
"""
df_datamart_purchaseorders = get_sql_dataframe(sql_fact_purchaseorders, **mysql_datamart_args)
df_datamart_purchaseorders.head(10)

,Vendor Name,Product Number,Total Due Amount
0,Advanced Bicycles,HJ-1213,1228.6401
1,Advanced Bicycles,HJ-1220,1228.6401
2,Advanced Bicycles,HJ-1420,1546.7758
3,Advanced Bicycles,HJ-1428,1546.7758
4,Advanced Bicycles,HJ-3410,1546.7758
5,Advanced Bicycles,HJ-3416,1546.7758
6,Advanced Bicycles,HJ-3816,1277.0872
7,Advanced Bicycles,HJ-3824,1277.0872
8,Advanced Bicycles,HJ-5161,1277.0872
9,Advanced Bicycles,HJ-5162,1509.6709


In [40]:
# Query 2: By vendor, showing quarterly total amount due:

sql_fact_purchaseorders = """
SELECT 
v.name AS 'Vendor Name', 
dt.calendar_year_qtr AS 'Year Quarter',
SUM(po.totaldue) AS 'Total Due Amount'
FROM fact_purchaseorders AS po
INNER JOIN dim_vendor AS v ON v.vendor_key = po.vendor_key
INNER JOIN dim_date as dt ON dt.date_key = po.order_date_key
GROUP BY v.vendor_key, dt.calendar_year_qtr
ORDER BY v.name, dt.calendar_year_qtr
"""
df_datamart_purchaseorders = get_sql_dataframe(sql_fact_purchaseorders, **mysql_datamart_args)
df_datamart_purchaseorders.head(10)

,Vendor Name,Year Quarter,Total Due Amount
0,Advanced Bicycles,2001Q2,601.3442
1,Advanced Bicycles,2002Q1,2475.2312
2,Advanced Bicycles,2002Q3,1297.9716
3,Advanced Bicycles,2002Q4,3326.2045
4,Advanced Bicycles,2003Q2,521.3468
5,Advanced Bicycles,2003Q3,10634.3880
6,Allenson Cycles,2001Q2,9776.2665
7,Allenson Cycles,2002Q1,9776.2665
8,Allenson Cycles,2002Q3,9776.2665
9,Allenson Cycles,2002Q4,9776.2665


In [41]:
########### BRONZE ############################
#Creating bronze streaming:
remove_directory_tree(purchase_orders_output_bronze)
df_purchase_orders_bronze = (
    spark.readStream \
    .option("schemaLocation", purchase_orders_output_bronze) \
    .option("maxFilesPerTrigger", 1) \
    .option("multiLine", "true") \
    .json(purchase_orders_stream_dir)
)

df_purchase_orders_bronze.isStreaming

True

In [42]:
#writing streaming data to parquet file
purchase_orders_checkpoint_bronze = os.path.join(purchase_orders_output_bronze, '_checkpoint')

purchase_orders_bronze_query = (
    df_purchase_orders_bronze
    # adding current timestamp and filename columns to trace.
    .withColumn("receipt_time", current_timestamp())
    .withColumn("source_file", input_file_name())
    
    .writeStream \
    .format("parquet") \
    .outputMode("append") \
    .queryName("purchase_orders_bronze")
    .trigger(availableNow = True) \
    .option("checkpointLocation", purchase_orders_checkpoint_bronze) \
    .option("compression", "snappy") \
    .start(purchase_orders_output_bronze)
)

In [43]:
#unit test
print(f"Query ID: {purchase_orders_bronze_query.id}")
print(f"Query Name: {purchase_orders_bronze_query.name}")
print(f"Query Status: {purchase_orders_bronze_query.status}")

Query ID: 4e93ebc9-29e5-43a3-814c-e86d3c178720
Query Name: purchase_orders_bronze
Query Status: {'message': 'Initializing sources', 'isDataAvailable': False, 'isTriggerActive': False}


In [44]:
purchase_orders_bronze_query.awaitTermination()

In [45]:
#checking json files loaded.
os.listdir(purchase_orders_output_bronze)

['.part-00000-6f00b7bf-fcbc-4e96-984a-b9d5606f7e47-c000.snappy.parquet.crc',
 'part-00000-6a32a411-14d3-4616-ae5b-05c5b07a97f6-c000.snappy.parquet',
 '_checkpoint',
 'part-00000-1342150b-3e05-48cb-a2e8-8da69cf3ade7-c000.snappy.parquet',
 '.part-00000-1342150b-3e05-48cb-a2e8-8da69cf3ade7-c000.snappy.parquet.crc',
 '_spark_metadata',
 'part-00000-6f00b7bf-fcbc-4e96-984a-b9d5606f7e47-c000.snappy.parquet',
 '.part-00000-6a32a411-14d3-4616-ae5b-05c5b07a97f6-c000.snappy.parquet.crc']

In [46]:
df_purchase_orders_bronze_stream = (
    spark.readStream
        .format("parquet")
        .load(purchase_orders_output_bronze)
)


In [47]:
df_purchase_orders_bronze_stream.isStreaming


True

In [48]:
########### SILVER ############################
# SILVER LAYER:
# Read streaming Bronze purchase order data, join with static dimension tables,
# replace natural keys with surrogate keys, and write the integrated fact data
# to the Silver layer using structured streaming (Lab 6 pattern).

#loading dim_date, dim_products, dim_vendors, and dim_employees (persisted dimension tables)
#from the Lakehouse to join with bronze facts.

df_dim_date = spark.table(f"{dest_database}.dim_date")
df_dim_products = spark.table(f"{dest_database}.dim_products")
df_dim_vendors = spark.table(f"{dest_database}.dim_vendors")
df_dim_employees = spark.table(f"{dest_database}.dim_employees")

In [49]:
#preparing role-playing dimension primary and business keys; order dates are different formats so cannot use alias
df_dim_order_date = df_dim_date.select(
    col("date_key").cast("long").alias("order_date_key"),
    col("month_of_year").cast("int").alias("month_of_year"),
    col("month_name").alias("month_name")
)

df_dim_ship_date = df_dim_date.select(
    col("date_key").alias("shipped_date_key")
)

In [50]:
#reading bronze and joining, don't need to cast since already int
#time format must be changed for dim_date to be used in streaming so using pyspark functions
from pyspark.sql.functions import from_unixtime, date_format

df_purchase_orders_silver = (
    df_purchase_orders_bronze_stream
       .join(
            df_dim_order_date,
            date_format(
                from_unixtime(col("OrderDate") / 1000),
                "yyyyMMdd").cast("long") == col("order_date_key"),
            "left"

        )
        .join(
            df_dim_products,
            col("ProductID") == df_dim_products.product_id,
            "left"
        )
        .join(
            df_dim_vendors,
            col("VendorID") == df_dim_vendors.vendor_id,
            "left"
        )
        .join(
            df_dim_employees,
            col("EmployeeID") == df_dim_employees.employee_id,
            "left"
        )
        .select(
            col("PurchaseOrderID").cast("long"),
            col("order_date_key").cast("long"),
            df_dim_order_date.month_of_year.cast("int"),
            df_dim_order_date.month_name,
            df_dim_products.product_key.cast("int"),
            df_dim_vendors.vendor_key.cast("int"),
            df_dim_employees.employee_key.cast("int"),
            col("OrderQty"),
            col("UnitPrice"),
            col("LineTotal"),
            col("TotalDue")
        )
)

#organizing columns
df_purchase_orders_silver = df_purchase_orders_silver.selectExpr(
    "PurchaseOrderID",
    "order_date_key",
    "month_of_year",
    "month_name",
    "product_key",
    "vendor_key",
    "employee_key",
    "OrderQty",
    "UnitPrice",
    "LineTotal",
    "TotalDue"
)


In [51]:
df_purchase_orders_silver.printSchema()

root
 |-- PurchaseOrderID: long (nullable = true)
 |-- order_date_key: long (nullable = true)
 |-- month_of_year: integer (nullable = true)
 |-- month_name: string (nullable = true)
 |-- product_key: integer (nullable = true)
 |-- vendor_key: integer (nullable = true)
 |-- employee_key: integer (nullable = true)
 |-- OrderQty: long (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- LineTotal: double (nullable = true)
 |-- TotalDue: double (nullable = true)



In [52]:
df_purchase_orders_silver.isStreaming

True

In [53]:
spark.read.parquet(purchase_orders_output_bronze) \
    .select("PurchaseOrderID", "OrderDate") \
    .show()

+---------------+-------------+
|PurchaseOrderID|    OrderDate|
+---------------+-------------+
|              1| 990057600000|
|              9|1010966400000|
|            281|1047427200000|
+---------------+-------------+



In [54]:
#writing the transformed streaming data to the data lakehouse
purchase_orders_checkpoint_silver = os.path.join(
    purchase_orders_output_silver, "_checkpoint"
)
purchase_orders_silver_query = (
    df_purchase_orders_silver
        .writeStream
        .format("parquet")
        .outputMode("append")
        .queryName("purchase_orders_silver")
        .trigger(availableNow=True)
        .option("checkpointLocation", purchase_orders_checkpoint_silver)
        .option("compression", "snappy")
        .start(purchase_orders_output_silver)
)

purchase_orders_silver_query.awaitTermination()

In [55]:
spark.read.parquet(purchase_orders_output_silver) \
    .select("PurchaseOrderID", "order_date_key", "month_of_year", "month_name") \
    .show()

+---------------+--------------+-------------+----------+
|PurchaseOrderID|order_date_key|month_of_year|month_name|
+---------------+--------------+-------------+----------+
|              9|      20020113|            1|   January|
|            281|      20030311|            3|     March|
|              1|      20010516|            5|       May|
+---------------+--------------+-------------+----------+



In [56]:
#stream process check
wait_until_stream_is_ready(purchase_orders_silver_query, 1)

The stream has processed 1 batchs


In [57]:
df_purchase_orders_silver_stream = (
    spark.readStream
        .format("parquet")
        .load(purchase_orders_output_silver)
)

In [58]:
df_purchase_orders_silver_stream.isStreaming


True

In [59]:
#unit test to implement query monitoring
print(f"Query ID: {purchase_orders_silver_query.id}")
print(f"Query Name: {purchase_orders_silver_query.name}")
print(f"Query Status: {purchase_orders_silver_query.status}")

Query ID: 037ac684-b5d7-4ef9-a737-bd7ad39e3d3a
Query Name: purchase_orders_silver
Query Status: {'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}


In [60]:
########### GOLD ############################
# This section mirrors Lab 6, Section 6 Gold. Like in Lab 6, the Silver-level fact data is aggregated, 
#joint to dimension tables, and compute monthly product metrics.
# For this project, AdventureWorks does not include Product Category in the dim_products table, 
# so Product Name is used as the dimension.

#Define query to create business report.
from pyspark.sql.functions import count

df_purchase_orders_by_product_gold = (
    df_purchase_orders_silver_stream
        .groupBy("month_of_year", "month_name", "product_key")
        .agg(count("*").alias("product_count"))
)


In [61]:
# writing the streaming data to a parquet file in "complete" mode
orders_gold_query = (
    df_purchase_orders_by_product_gold.writeStream \
    .format("memory") \
    .outputMode("complete") \
    .queryName("df_fact_purchase_orders_by_product")
    .trigger(availableNow=True)
    .start()
)

orders_gold_query.awaitTermination()

In [62]:
wait_until_stream_is_ready(orders_gold_query, 1)
orders_gold_query.awaitTermination()

The stream has processed 1 batchs


In [63]:
spark.sql("SELECT * FROM df_fact_purchase_orders_by_product").show()

+-------------+----------+-----------+-------------+
|month_of_year|month_name|product_key|product_count|
+-------------+----------+-----------+-------------+
|            3|     March|        141|            1|
|            5|       May|          1|            1|
|            1|   January|        105|            1|
+-------------+----------+-----------+-------------+



In [64]:
#creating the final selection
df_fact_purchase_orders_by_product_full = spark.sql("""
    SELECT
        month_of_year,
        month_name,
        product_key,
        product_count
    FROM df_fact_purchase_orders_by_product
""")

In [65]:
#load final table and display results in table
df_fact_purchase_orders_by_product_full.write \
    .mode("overwrite") \
    .saveAsTable(f"{dest_database}.fact_purchase_orders_by_product")

final_df_pd = spark.sql(f" SELECT month_name AS Month, \
        product_key AS Product, \
        product_count AS Product_Count \
    FROM {dest_database}.fact_purchase_orders_by_product \
    ORDER BY month_of_year, Product_Count DESC ").toPandas()

final_df_pd

# This final Gold fact table displays the number of purchase-order line items
# per product, by month, enabling business analysis
# of purchasing activity over time (by showing how many times each product appears 
# in purchase orders for each month.)

,Month,Product,Product_Count
0,January,105,1
1,March,141,1
2,May,1,1


In [66]:
##additional query using more dimensions:
#query from silver
df_purchase_orders_silver_batch = spark.read.parquet(purchase_orders_output_silver)

#joining dimensions to aggregate
df_fact_purchase_orders_by_vendor_employee = (
    df_purchase_orders_silver_batch
        .join(df_dim_vendors, "vendor_key")
        .join(df_dim_employees, "employee_key")
        .groupBy(
            "month_name",
            "month_of_year",
            "vendor_key",
            "employee_key"
        )
        .agg(
            count("*").alias("order_count"),
            sum("TotalDue").alias("total_spend")
        )
)
#vendors table
df_fact_purchase_orders_by_vendor_employee.write \
    .mode("overwrite") \
    .saveAsTable(f"{dest_database}.fact_purchase_orders_by_vendor_employee")

#vendors query
vendor_employee_report = spark.sql(f"""
    SELECT
        month_name AS Month,
        vendor_key AS Vendor,
        employee_key AS Employee,
        order_count AS Orders,
        total_spend AS Total_Spend
    FROM {dest_database}.fact_purchase_orders_by_vendor_employee
    ORDER BY month_of_year, Total_Spend DESC
""").toPandas()

vendor_employee_report

# This report summarizes purchase-order activity by vendor and employee,
# by each month, including number of orders and total spending.

,Month,Vendor,Employee,Orders,Total_Spend
0,January,74,261,1,767.0528
1,March,1,244,1,158.0608
2,May,83,244,1,222.1492


In [67]:
spark.stop()